In [1]:
# Download all packages 
import pickle
import detritalpy.detritalFuncs as dFunc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from random import choices
from random import randrange
from random import randint
import random
import time
from drs import drs

In [2]:
# Create a list of all source age distributions
all_sources = ["New 0.15-0.25 Sources.p","New 0.25-0.35 Sources.p","New 0.35-0.45 Sources.p",
               "New 0.45-0.55 Sources.p","New 0.55-0.65 Sources.p","New 0.65-0.75 Sources.p",
               "New 0.75-0.85 Sources.p","New 0.85-0.95 Sources.p","New 0.95-0.99 Sources.p"]

In [ ]:
# Start by looping through the sources
for source in all_sources:

    # Lets load in the original sources
    sims_data = pickle.load(open('New 0.95-0.99 Sources.p','rb'))

    # Lets load all of the data into a dataframe
    IDs = []
    KDEs = []
    sources = []
    for entry in sims_data:
        ID = entry
        IDs.append(ID)
        KDEs.append(sims_data[entry][0])
        sources.append(sims_data[entry][1])

        
    # Put into a dataframe
    df = pd.DataFrame()
    df['ID'] = IDs
    df['KDE'] = KDEs
    df['Sources'] = sources

    ###########################################
    # Presets
    bw = 15
    xdif = 1
    num_samples = 10
    hit_vals = np.round(np.logspace(1,3,num=35),0)
    ###########################################
    # start a timer
    start_time = time.time()
    # t will be the number of sources
    for t in range(0,8):
        # Create storage
        final_synth_dict = {}
        # Grab the KDE we will be working with
        index_KDE = [df.loc[t]['KDE']]
        index_source = df.loc[t]['Sources']
        # Create a list of ages from a given probability
        index_age = []
        for r in range(0,500):
            pick = np.random.choice(np.arange(0,4501), p = np.sum(index_KDE,axis = 0)/np.sum(index_KDE))
            index_age.append(pick)
        init_KDE_age, init_KDE = dFunc.KDEcalcAges_2(index_age,x1=0,x2=4500,xdif=xdif,bw=bw,cumulative = False)
        # w will be the number of grains utilized
        for w in hit_vals:
            total_KDE_list = []
            w = int(w)
            for i in range(0,num_samples):
                # INTRODUCE PROPORTION VARIABILITY ######################################
                new_index_sources = []
                # ADJUSTING SOURCES #####################################################
                
                drs_coeff = drs(len(index_source),5)
                for i in range(len(index_source)):
                    ind_source = index_source[i]
                    ind_drs = drs_coeff[i]
                    new_ind_source = []
                    for var in ind_source:
                        new_var = var*ind_drs
                        new_ind_source.append(new_var)
                    new_ind_source = np.asarray(new_ind_source)
                    new_index_sources.append(new_ind_source)
                new_index_KDE = sum(new_index_sources)
                
                ########################################################################
                new_index_KDE = [np.asarray(new_index_KDE)]
                #########################################################################
                
                source_dist = [np.random.choice(init_KDE_age, size = w, p = np.sum(new_index_KDE, axis = 0)/np.sum(new_index_KDE))]
                source_KDE_age, source_KDE = dFunc.KDEcalcAges_2(source_dist, x1=0, x2=4500, xdif=xdif, bw=bw, cumulative=False)
                source_KDE = source_KDE[0]
                total_KDE_list.append(source_KDE)
            final_synth_dict[t,w] = total_KDE_list
            end_time = time.time()
            total_time = end_time-start_time
            print('Finished with sample',t,'analyses',w)
            print('Total Run Time:',total_time)

        # Create the saving label
        label = str(source[4:13]) + ' - ' + str(t) + ' Variable Sources.p'
        # Save all of the synthetic data
        pickle.dump(final_synth_dict,open(label,'wb'))